## This notebook converts the original Income, COMPAS and Credit datasets into a usable format for our fairCorrect scripts.

In [ ]:
# Necessary packages

import pandas as pd
import numpy as np

# Files get loaded from fairCorrect github repository
url = 'https://raw.githubusercontent.com/vladoxNCL/fairCorrect/master/Datasets/'

# Specify where to save the processed files as savepath
savepath = 'path/to/file/'

## COMPAS cleanup

In [ ]:
df = pd.read_csv(url + 'compas-scores-two-years.csv', header=0)

# select relevant columns
df =df[['age', 'c_charge_degree', 'race', 'sex', 'priors_count', 'days_b_screening_arrest',
        'two_year_recid', 'c_jail_in', 'c_jail_out']]

# Remove missing values
df = df.dropna()

# Make PA binary
df.race = [0 if r != 'Caucasian' else 1 for r in df.race]

# Make two_year_recid the label
df = df.rename({'two_year_recid': 'label'}, axis=1)

# reverse label for consistency with function defs: 1 means no recid (good), 0 means recid (bad)
df.label = [0 if l == 1 else 1 for l in df.label]

# convert jailtime to days
df['jailtime'] = (pd.to_datetime(df.c_jail_out) - pd.to_datetime(df.c_jail_in)).dt.days

#drop jail in and out dates
df = df.drop(['c_jail_in', 'c_jail_out'], axis=1)

# Encode sex as 0 for female, 1 for male
df.sex = [0 if s != 'Male' else 1 for s in df.sex]

# M: misconduct, F: felony
df.c_charge_degree = [0 if s == 'M' else 1 for s in df.c_charge_degree]

# Uncomment bottom line to save clean csv
#df.to_csv(savepath + 'compas_onehot.csv', index=False)

## German Credit cleanup

In [ ]:
df = pd.read_csv(url + 'german.csv', header=None)

# Data needed column names
df.columns = ['checking', 'duration', 'credit_history', 'purpose', 'credit_amount',
              'savings', 'employment', 'inst_rate', 'personal_status', 'other_debtors',
              'residence_time', 'property', 'age', 'other_inst', 'housing', 'num_credits',
              'job', 'dependants', 'phone', 'foreigner', 'label']

# Turn criptic values into interpretable form
df = df.replace({'checking': {'A11': 'check_low', 'A12': 'check_mid', 'A13': 'check_high',
                              'A14': 'check_none'},
                 'credit_history': {'A30': 'debt_none', 'A31': 'debt_noneBank',
                                    'A32': 'debt_onSchedule','A33': 'debt_delay',
                                    'A34': 'debt_critical'},
                 'purpose': {'A40': 'pur_newCar', 'A41': 'pur_usedCar',
                             'A42': 'pur_furniture', 'A43': 'pur_tv',
                             'A44': 'pur_appliance', 'A45': 'pur_repairs',
                             'A46': 'pur_education', 'A47': 'pur_vacation',
                             'A48': 'pur_retraining', 'A49': 'pur_business',
                             'A410': 'pur_other'},
                 'savings': {'A61': 'sav_small', 'A62': 'sav_medium', 'A63': 'sav_large',
                             'A64': 'sav_xlarge', 'A65': 'sav_none'},
                 'employment': {'A71': 'emp_unemployed', 'A72': 'emp_lessOne',
                                'A73': 'emp_lessFour', 'A74': 'emp_lessSeven',
                                'A75': 'emp_moreSeven'},
                 'other_debtors': {'A101': 'debtor_none', 'A102': 'debtor_coApp',
                                   'A103': 'debtor_guarantor'},
                 'property': {'A121': 'prop_realEstate', 'A122': 'prop_agreement',
                              'A123': 'prop_car', 'A124': 'prop_none'},
                 'other_inst': {'A141': 'oi_bank', 'A142': 'oi_stores', 'A143': 'oi_none'},
                 'housing': {'A151': 'hous_rent', 'A152': 'hous_own', 'A153': 'hous_free'},
                 'job': {'A171': 'job_unskilledNR', 'A172': 'job_unskilledR',
                         'A173': 'job_skilled', 'A174': 'job_highSkill'},
                 'phone': {'A191': 0, 'A192': 1},
                 'foreigner': {'A201': 1, 'A202': 0},
                 'label': {2: 0}})

# More criptic values translating
df['status'] = np.where(df.personal_status == 'A91', 'divorced',
                        np.where(df.personal_status == 'A92', 'divorced', 
                                 np.where(df.personal_status == 'A93', 'single',
                                          np.where(df.personal_status == 'A95', 'single',
                                                   'married'))))
# Translate gender values
df['gender'] = np.where(df.personal_status == 'A92', 0,
                        np.where(df.personal_status == 'A95', 0,
                                 1))

# Drop personal_status column
df = df.drop(['personal_status'], axis=1)

# One-hot encode categorical columns
col = ['checking', 'credit_history', 'purpose', 'savings', 'employment', 'other_debtors', 'property',
       'other_inst', 'housing', 'job', 'status']

for c in col:
    dummies = []
    dummies.append(pd.get_dummies(df[c]))
    df_dummies = pd.concat(dummies, axis = 1)
    df = pd.concat((df, df_dummies), axis = 1)
    df = df.drop([c], axis = 1)
    
# Uncomment bottom line to save clean csv
#df.to_csv(savepath + 'german_onehot.csv', index=False)

## Census Income cleanup

In [ ]:
# Get balancing feature to end column
df = pd.read_csv(url + 'adult.data', header=None)

# Assign names to columns
names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
         'occupation', 'relationship', 'race', 'sex', 'capital-gain',
         'capital-loss', 'hours-per-week', 'native-country', 'label']

df.columns = names

# Cleanup names from spaces
col = ['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'label']

for c in col:
    df[c] = df[c].map(str.strip)

# Replace ? character for NaN value
df = df.replace('?', np.nan)

# One-hot encode categorical variables
col = ['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'native-country']

for c in col:
    dummies = []
    dummies.append(pd.get_dummies(df[c]))
    df_dummies = pd.concat(dummies, axis = 1)
    df = pd.concat((df, df_dummies), axis = 1)
    df = df.drop([c], axis = 1)

# Assign sex and label binary values 0 and 1
df.sex = df.sex.replace('Male', 1)
df.sex = df.sex.replace('Female', 0)
df.label = df.label.replace('<=50K', 0)
df.label = df.label.replace('>50K', 1)

# Drop fnlwgt variable
df = df.drop(['fnlwgt'], axis=1)

# Uncomment bottom line to save clean csv
#df.to_csv(savepath + 'income_onehot.csv', index=False)